## kNNを使って欠損値を予測し補填

In [16]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [20]:
df = pd.read_csv('../section7/vgsales.csv')

# knnの学習データには、テストデータは含めない
# テストデータに対しても、学習データを使ったkNNで予測する
df[['Publisher']] = df[['Publisher']].fillna('NaN')
df.drop('Name', axis = 1, inplace = True)

target = 'Year'
X = df.drop(target, axis = 1)
y = df[target]

num_cols = X.select_dtypes(include=np.number).columns.to_list()
# ダミー変数
X = pd.get_dummies(X, drop_first = True)
# 機械学習でtrain, testを分ける必要がある実務の場合はtrainでのみfit
X[num_cols] = StandardScaler().fit_transform(X[num_cols])

In [23]:
train_indexes = df[~df['Year'].isna()].index
test_indexes = df[df['Year'].isna()].index

X_train, X_test = X.iloc[train_indexes], X.iloc[test_indexes]
y_train, y_test = y.iloc[train_indexes], y.iloc[test_indexes]

In [27]:
knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [28]:
X_test.head(1)

,Rank,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Platform_3DO,Platform_3DS,Platform_DC,Platform_DS,...,Publisher_Zushi Games,Publisher_bitComposer Games,Publisher_dramatic create,Publisher_fonfun,Publisher_iWin,Publisher_id Software,Publisher_imageepoch Inc.,Publisher_inXile Entertainment,"Publisher_mixi, Inc",Publisher_responDESIGN
179,-1.69472,4.892293,0.224302,-0.219159,3.510062,3.01776,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
neighbors = knn.kneighbors(X_test.head(1))
print(neighbors)
df.iloc[neighbors[1][0]]
# 特徴量の調整などを行ってない影響で、GlobalSalesを元に近さを算出しているため
# 予測値の妥当性は疑問符

(array([[0.47983803, 1.05095264, 1.53257282]]), array([[210, 239, 278]]))


,Rank,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
210,211,PS3,2007.0,Action,Sony Computer Entertainment,2.31,1.73,0.12,0.78,4.94
239,240,2600,1981.0,Platform,Activision,4.21,0.24,0.00,0.05,4.50
278,279,PS,1997.0,Action,Hasbro Interactive,3.79,0.27,0.00,0.11,4.16
